In [20]:
import pandas as pd
from sqlite3 import connect

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Создайте соединение с базой данных с помощью библиотеки sqlite3.**
**2. Получите схему таблицы test.**

In [21]:
con = connect('/content/drive/MyDrive/Школа 21/Проверка/project11/data/checking-logs.sqlite')

In [22]:
pd.read_sql('pragma table_info(test)', con=con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


**3. Получите только первые 10 строк таблицы test, чтобы проверить, как она выглядит.**

In [23]:
pd.read_sql('select * from test limit 10', con=con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


**4. Найдите среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.**
*   Для этого сджойните данные своей таблицы с данными таблицы deadlines
*   Период времени должен отображаться в часах.
*   Не учитывайте лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
*   Сохраните значение в датафрейме df_min с соответствующим uid.

In [24]:
row_sql = '''
    SELECT uid, min(strftime('%s', first_commit_ts) - deadlines)/60/60 AS "MIN(diff)"
    FROM
        test
    JOIN
        deadlines
    ON
        labname = labs
    WHERE
        labs <> 'project1'
'''

In [25]:
df_min = pd.read_sql(row_sql, con=con)
df_min

,uid,MIN(diff)
0,user_30,-202


**5. Выполните те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max**

In [26]:
row_sql = '''
    SELECT uid, max(strftime('%s', first_commit_ts) - deadlines)/60/60 AS "MAX(diff)"
    FROM
        test
    JOIN
        deadlines
    ON
        labname = labs
    WHERE
        labs <> 'project1'
'''

In [27]:
df_max = pd.read_sql(row_sql, con=con)
df_max

,uid,MAX(diff)
0,user_25,-2


**6. Выполните те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg**

In [28]:
row_sql = '''
    SELECT avg((strftime('%s', first_commit_ts) - deadlines)/60/60) AS "AVG(diff)"
    FROM
        test
    JOIN
        deadlines
    ON
        labname = labs
    WHERE
        labs <> 'project1'
'''

In [29]:
df_avg = pd.read_sql(row_sql, con=con)
df_avg

,AVG(diff)
0,-89.125


**7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого вам необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.**
*   Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews.

uid — это uid, существующие в таблице test.

avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.

pageviews — количество посещений ленты новостей одним пользователем.
*   Не учитывайте лабораторное задание project1.
*   Сохраните результаты в датафрейме views_diff.
*   Используйте метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [30]:
row_sql = f'''
    SELECT
        t.uid,
        pageviews,
        avg((strftime('%s', first_commit_ts) - deadlines)/60/60) AS avg_diff
    FROM 
        test as t
    JOIN
        deadlines
    ON
        labname = labs
    JOIN
       (SELECT uid, count() AS pageviews FROM pageviews GROUP BY uid) AS vpp
    ON 
        vpp.uid = t.uid
    WHERE
        labs != 'project1'
    GROUP BY 
        t.uid
'''

In [31]:
views_diff = pd.read_sql(row_sql, con=con)
views_diff

,uid,pageviews,avg_diff
0,user_1,28,-64.400000
1,user_10,89,-74.800000
2,user_14,143,-159.000000
3,user_17,47,-61.600000
4,user_18,3,-5.666667
5,user_19,16,-98.750000
6,user_21,10,-95.500000
7,user_25,179,-92.600000
8,user_28,149,-86.400000
9,user_3,317,-105.400000


In [32]:
views_diff.corr()

,pageviews,avg_diff
pageviews,1.000000,-0.279736
avg_diff,-0.279736,1.000000


**8. Закройте соединение.**

In [33]:
con.close()